## Import as .npy, export as a TTree in .root

In [1]:
import time
import numpy as np
import uproot


In [2]:
#input_filebase = 'bootstrap-9d-output-test1f'
#output_filename = 'bootstrap-9d-export1f.root'


#input_filebase = 'bootstrap-output-test1c'

input_filebase = 'bootstrap-9d-output-test1g-diagonal-cov'
output_filename = 'bootstrap-9d-export1g-diagonal-cov.root'

In [3]:
with open('%s.npy' % input_filebase,'rb') as f :
    train_pts = np.load(f)
    train_det_pts = np.load(f)
    true_pts = np.load(f)
    true_det_pts = np.load(f)
    bootstrap_weights = np.load(f)

In [4]:
nevts = len(train_pts)
nfeatures = len(train_pts[0,:])

print("\n\n train_pts:  shape %s ,  number of features in the data:  %d ,  nevts = %d \n\n" 
      % (str(train_pts.shape), nfeatures, nevts))



 train_pts:  shape (400000, 9) ,  number of features in the data:  9 ,  nevts = 400000 




In [5]:
nboot = len(bootstrap_weights)

print("\n\n bootstrap_weights:  shape %s ,  number of bootstrap samples:  %d\n\n" 
      % ( str(bootstrap_weights.shape), nboot) ) 



 bootstrap_weights:  shape (50, 400000) ,  number of bootstrap samples:  50




In [6]:
uproot4_branches = {}

for bi in range( nfeatures ) :
    uproot4_branches[ "x%d" % bi ] = "f4"

uproot4_branches[ "weight" ] = "f4"
uproot4_branches[ "dset_index"] = "i4"
    

In [7]:
uproot4_branches

{'x0': 'f4',
 'x1': 'f4',
 'x2': 'f4',
 'x3': 'f4',
 'x4': 'f4',
 'x5': 'f4',
 'x6': 'f4',
 'x7': 'f4',
 'x8': 'f4',
 'weight': 'f4',
 'dset_index': 'i4'}

In [8]:
%%time

with uproot.recreate(output_filename) as file :
    
    file.mktree( "boot_ttree", uproot4_branches, "Bootstrap ttree")
    
    for bi in range(nboot) :
        extend_dict = {}
        for fi in range(nfeatures) :
            extend_dict[ "x%d" % fi ] = train_pts[:,fi][bootstrap_weights[bi,:]<10]
        extend_dict["weight"] = bootstrap_weights[bi,:][bootstrap_weights[bi,:]<10]
        extend_dict["dset_index"] = bi * np.ones( len(bootstrap_weights[bi,:][bootstrap_weights[bi,:]<10]) )
        file["boot_ttree"].extend( extend_dict )



CPU times: user 24.7 s, sys: 957 ms, total: 25.7 s
Wall time: 25.7 s


In [9]:
bi=1
np.shape(train_pts[:,fi][bootstrap_weights[bi,:]<10])

(399993,)

In [10]:
np.shape(bootstrap_weights[bi,:][bootstrap_weights[bi,:]<10])

(399993,)